In [12]:
import sys
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
import math
from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.clustering import LDA, LDAModel
from sklearn.neighbors import NearestNeighbors
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
import textwrap
from datetime import date, datetime

In [85]:
f = sc.textFile("file:///bgdt/yellow.50000.csv") # file:// for local files (default hdfs)
header = f.first()
[ str(i) + ' ' + x for i, x in enumerate(header.split(",")) ]

[u'0 VendorID',
 u'1 tpep_pickup_datetime',
 u'2 tpep_dropoff_datetime',
 u'3 passenger_count',
 u'4 trip_distance',
 u'5 pickup_longitude',
 u'6 pickup_latitude',
 u'7 RatecodeID',
 u'8 store_and_fwd_flag',
 u'9 dropoff_longitude',
 u'10 dropoff_latitude',
 u'11 payment_type',
 u'12 fare_amount',
 u'13 extra',
 u'14 mta_tax',
 u'15 tip_amount',
 u'16 tolls_amount',
 u'17 improvement_surcharge',
 u'18 total_amount']

In [86]:
# def hour2ts(h):
#     if h in {}:
        
#     elif

dtf = "%Y-%m-%d %H:%M:%S"  # date format

univ = f.filter(lambda x: x != header)\
.map(lambda l: l.split(","))\
.map(lambda c:
     {        'pt': c[ 1],
       'isWeekend': int(datetime.strptime(c[1],dtf).isoweekday()>5),
                    # isoweekday() returns Monday as 1, Sundey as 7
              'ph': datetime.strptime(c[1],dtf).hour//2,
              'dt': c[ 2],
             'len': (datetime.strptime(c[2],dtf) -
                     datetime.strptime(c[1],dtf)).seconds // 60,
                    # actual trip duration in minutes
             'dst': float(c[ 4]),
              'po': float(c[ 5]),
              'pa': float(c[ 6]),
              'do': float(c[ 9]),
              'da': float(c[10]),
               '$': c[11], # payment type: 1 is credit card (tip recorded)
            'fare': float(c[12]),
              'ex': float(c[13]),
             'tip': float(c[15]),
            'toll': float(c[16]),
      })\
.filter(lambda x: x['$'] == '1' )\
.filter(lambda x: x['len'] >= 2 )\
.map(lambda c:
     { 'isWeekend': c['isWeekend'],
              'ph': c['ph'],
             'len': c['len'],
      'isHighHpay': int((c['tip']*60/c['len'])>=12),
             'dst': c['dst'],
              'po': c['po'],
              'pa': c['pa'],
              'do': c['do'],
              'da': c['da'],
            'fare': c['fare'],
              'ex': c['ex'],
            'toll': c['toll']
      })\
.cache()

univ.take(1)

[{'da': 40.7442512512207,
  'do': -73.99568939208984,
  'dst': 2.43,
  'ex': 0.5,
  'fare': 12.0,
  'isHighHpay': 1,
  'isWeekend': 0,
  'len': 15,
  'pa': 40.7635383605957,
  'ph': 0,
  'po': -73.96932983398438,
  'toll': 0.0}]

In [87]:
nrCards = univ.count()
print("%d credit card trips" % (nrCards) )

27679 credit card trips


In [121]:
#preped = univ.map(lambda l: (l['dst'], l['tip']) )
moment = 'finished'

if moment == 'before': 
    print("use before")
    preped = univ.map(lambda l:
                      LabeledPoint(l['isHighHpay'],
                                   [l['isWeekend'],
                                    l['po'],
                                    l['pa'],
                                    l['ph'],
                                    l['ex']]))
elif moment == 'finished':
    print("use finished")
    preped = univ.map(lambda l:
                      LabeledPoint(l['isHighHpay'],
                                   [l['isWeekend'],
                                    l['po'],
                                    l['pa'],
                                    l['ph'],
                                    l['ex'],
                                    l['dst'],
                                    l['toll'],
                                    l['fare'],
                                    l['len'],
                                    l['do'],
                                    l['da']
                                   ]))
else:
    print("not yet")
preped.take(1)

use finished


[LabeledPoint(1.0, [0.0,-73.969329834,40.7635383606,0.0,0.5,2.43,0.0,12.0,15.0,-73.9956893921,40.7442512512])]

In [122]:
trRDD, vaRDD, teRDD = preped.randomSplit([6,2,2], seed=0L)

print("  training RDD:", trRDD.take(1))
print("validation RDD:", vaRDD.map(lambda x: x.features ).take(1))

('  training RDD:', [LabeledPoint(1.0, [0.0,-74.0043029785,40.7422409058,0.0,0.5,3.7,0.0,14.0,14.0,-74.0073623657,40.7069358826])])
('validation RDD:', [DenseVector([0.0, -73.9821, 40.7747, 0.0, 0.5, 1.7, 0.0, 7.0, 4.0, -73.9709, 40.7967])])


In [123]:
model = RandomForest.trainClassifier(trRDD, numClasses=2,
                                     categoricalFeaturesInfo = {},
                             numTrees=50, featureSubsetStrategy="sqrt",
                            impurity="gini", maxDepth=4, maxBins=32, seed = 1)
# gini is faster

In [124]:
model

TreeEnsembleModel classifier with 50 trees

In [125]:
preds = model.predict(vaRDD.map(lambda x: x.features ))

In [126]:
labelsAndPredictions = vaRDD.map(lambda lp: lp.label).zip(preds)


In [127]:
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(vaRDD.count())
print('Test Accuracy = %f percent' % (100-round(testErr,7)*100))
# print('Learned classification forest model:')
# print(model.toDebugString()) # messy

Test Accuracy = 75.432530 percent


In [128]:
nActuNotHigh = labelsAndPredictions.filter(lambda lp: lp[0] == 0 ).count()
nActuHigh    = labelsAndPredictions.filter(lambda lp: lp[0] == 1 ).count()
nPredNotHigh = labelsAndPredictions.filter(lambda lp: lp[1] == 0 ).count()
nPredHigh    = labelsAndPredictions.filter(lambda lp: lp[1] == 1 ).count()

print("Baseline : %f     " % (float(max(nActuNotHigh,nActuHigh)) / float(nActuNotHigh+nActuHigh)))
print("Actual  0: %9d    1: %9d     " % (nActuNotHigh, nActuHigh) )
print("Predict 0: %9d    1: %9d     " % (nPredNotHigh, nPredHigh) )

Baseline : 0.690585     
Actual  0:      1699    1:      3792     
Predict 0:      1000    1:      4491     


In [ ]:
# Note: for unfiltered data,
# 69 -before-> 72% -finish-> 75%
#
#